In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/finance-mcq-question/finance_MCQ_topic.csv")
df 

In [ ]:
! pip install -q -U bitsandbytes
! pip install -q -U git+https://github.com/huggingface/transformers.git 
! pip install -q -U git+https://github.com/huggingface/peft.git
#pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
! pip install -q -U accelerate
! pip install -q -U datasets

In [ ]:
from datasets import load_dataset
data = load_dataset("csv", data_files="/kaggle/input/finance-mcq-question/finance_MCQ_topic.csv")
data #= data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_name = "bigscience/bloom-3b"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0})

In [ ]:
def generate_prompt(data_point):
    return f""" Examine the given options thoroughly and determine the optimal choice that accurately addresses the following multiple-choice question.
    ### Question:
    {data_point['Question']}
    ### Option A:
    {data_point['Option A']}
    ### Option B:
    {data_point['Option B']}
    ### Option C:
    {data_point['Option C']}
    ### Option D:
    {data_point['Option D']}
    ### Correct Ans of the above MCQ question is :
    {data_point['Answer']}
    ### Above answer is correct because : 
    {data_point['Explanation']}"""

mapped_qa_dataset = data.map(lambda samples: tokenizer(generate_prompt(samples)))

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16, 
    lora_alpha=64, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, #
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
pip install wandb

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_qa_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=5,
        warmup_steps=100,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

In [ ]:
HUGGING_FACE_USER_NAME = "aayushi135"

In [ ]:
model_name = "bloom3b_finetuned_1000"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

In [ ]:
from peft import LoraConfig
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch

base_model_name_or_path = "bigscience/bloom-3b"
repo_name = f"{HUGGING_FACE_USER_NAME}/{model_name}"

config = LoraConfig.from_pretrained(repo_name)


model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
# Load the LoRA model
inference_model = PeftModel.from_pretrained(model, repo_name)   # <-- e

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [ ]:
context = "Answer the following question"
question = "What is a stock market?"

In [ ]:
def make_inference(data_point):
    batch = tokenizer(f"""
    ### Question:
    {data_point['Questions']}
    ### Option A:
    {data_point['Option A']}
    ### Option B:
    {data_point['Option B']}
    ### Option C:
    {data_point['Option C']}
    ### Option D:
    {data_point['Option D']}
    ### Correct Ans:""", return_tensors='pt')
    with torch.cuda.amp.autocast():
        output_tokens = inference_model.generate(**batch, max_new_tokens=200)
    print((tokenizer.decode(output_tokens[0], skip_special_tokens=True)))
    #display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
df1 = pd.read_csv("/kaggle/input/finance/test.csv")
df1

In [ ]:
df1 = pd.read_csv("/kaggle/input/finance/CA_exam.csv")
df1.head()

In [ ]:
l = []
for i in range(0,len(df1))  :
  print(i)
  l.append(make_inference(df1.iloc[i,:]))

In [ ]:
l = []
for i in range(50,len(df1))  :
  print(i)
  l.append(make_inference(df1.iloc[i,:]))

In [ ]:
df.iloc[0,0]

In [ ]:
l = []
for i in range(0,50)  :
  print(i)
  l.append(make_inference(df1.iloc[i,:]))

In [ ]:
question = df1.iloc[2,1]
print(question)
make_inference(question)

In [ ]:
question = df1.iloc[1,1]
print(question)
make_inference(question)

In [ ]:
def make_inference2(question):
    batch = tokenizer(f"### Instruction:find out the sentiment for following sentence only in one word do not generate any other examples\n\n### Input:{question}\n\n### Response\n", return_tensors='pt')
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=200)
    print((tokenizer.decode(output_tokens[0], skip_special_tokens=True)))
    #display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
question = df1.iloc[0,1]
print(question)
make_inference2(question)

In [ ]:
question = "Our Company is not getting any work"
print(question)
make_inference(question)

In [ ]:
def make_inference1(question):
    batch = tokenizer(f"### Instruction:Answer the following question \n\n### Input:{question}\n\n### Response\n", return_tensors='pt')
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=200)
    print((tokenizer.decode(output_tokens[0], skip_special_tokens=True)))
    #display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))
    
question = "What is stock market ?"
make_inference1(question)

In [ ]:
for i in range(0,100) :
    print(i)
    df1.iloc[i,3] = make_inference(df1.iloc[i,1])

In [ ]:
df1 = pd.read_csv("/kaggle/input/finance/sentiment.csv")
df1

In [ ]:
df1['pred'] = df1['1']
df1

In [ ]:
prompt = df1.iloc[2,1]
result_length = 50
inputs = tokenizer(prompt, return_tensors="pt")
print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_length=result_length
                      )[0]))

In [ ]:
for i in range(0,100) :
    print(i)
    prompt = "Generate the sentiment for following sentence : " + df1.iloc[i,1]

In [ ]:
l = []
for i in range(0,100) :
    print(i)
    prompt = "Generate the sentiment for following sentence : " + df1.iloc[i,1]
    inputs = tokenizer(prompt, return_tensors="pt")
    l.append(tokenizer.decode(model.generate(inputs["input_ids"])[0]))

In [ ]:
l

In [ ]:
df1['pred'] = df['0'].apply(lambda x : make_inference(x))

In [ ]:
df1.to_csv("bloom_3b_generated.csv")

In [ ]:
text = f"""### Instruction:\n{context}\n\n### Input:\n{question} ### Response:\n"""
#device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
HUGGING_FACE_USER_NAME = "aayushi135"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "qlora_finance"

model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)